In [ ]:
#clone to get dataset
!git clone https://github.com/jain-abhinav02/Toxic_Spans_Detection.git

Cloning into 'Toxic_Spans_Detection'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 150 (delta 72), reused 108 (delta 48), pack-reused 0
Receiving objects: 100% (150/150), 1.71 MiB | 2.38 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [ ]:
import pandas as pd
import textblob
import json
from nltk.tokenize import word_tokenize
import nltk
     

In [ ]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

def f1(preds,trues):
    if len(trues) == 0:
        return 1. if len(preds) == 0 else 0.
    if len(preds) == 0:
        return 0.
    predictions_set = set(preds)
    gold_set = set(trues)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [ ]:

def avg_f1(preds,trues):
    avg_f1_total = 0.0
    for pred,true in zip(preds,trues):
        avg_f1_total += f1(pred,true)
    return avg_f1_total/len(preds)
     


In [ ]:

test_set = pd.read_csv("/content/Toxic_Spans_Detection/datasets/tsd_test.csv")
test_set['spans'] = test_set['spans'].apply(lambda x : json.loads(x))
train_set = pd.read_csv("/content/Toxic_Spans_Detection/datasets/tsd_train.csv")
train_set['spans'] = train_set['spans'].apply(lambda x : json.loads(x))
dataset = test_set.append(train_set,ignore_index=True)
dataset['text'] = dataset['text'].apply(lambda x : x.lower())
     

In [ ]:
#Hate Words list

hate_words = pd.read_csv("/content/bad-words.csv")['jigaboo'].to_list()

In [ ]:
indices_all = []
for index,row in dataset.iterrows():
    text = row['text']
    start = 0
    indices = []
    for word in word_tokenize(text):
        word_blob = textblob.TextBlob(word)
        if word_blob.polarity < 0:
            start_index = text[start:].find(word)
            indices += list(range(start + start_index,start + start_index+len(word)))
            start += start_index+len(word)
    indices_all.append(indices)
     

toxic_f1 = avg_f1(indices_all,dataset['spans'].to_numpy())
     

print("F1 : %f"%(toxic_f1))

F1 : 0.390830


In [ ]:
indices_all = []
for index,row in dataset.iterrows():
    text = row['text']
    start = 0
    indices = []
    for word in word_tokenize(text):
        word_blob = textblob.TextBlob(word)
        if word_blob.polarity < 0 or word in hate_words:
            start_index = text[start:].find(word)
            indices += list(range(start + start_index,start + start_index+len(word)))
            start += start_index+len(word)
    indices_all.append(indices)
     

toxic_f1 = avg_f1(indices_all,dataset['spans'].to_numpy())
     

print("F1 : %f"%(toxic_f1))

F1 : 0.425147
